## [Demo] Working with PSANA inside Jupyter lab notebook

This notebook starts a Dask scheduler that serves as a bridge to share data between PSANA and Jupyter Lab. Using the `slurm` package, users can submit a PSANA job and return results to be manipulated within the notebook. 

This notebook is able to run the example shown in https://confluence.slac.stanford.edu/display/PSDMInternal/Small+H5

Note: `slurm_magic` version from https://github.com/muammar/slurm-magic. 

In [1]:
print("remove # to clean")
!rm -rf *.json *.log .tmp *.out dask-worker-space/

remove # to clean


Load the `slurm_magic` package

In [2]:
%load_ext slurm_magic
%squeue -u melkhati

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)


We proceed to start a scheduler in the login node to be our bridge to share data. 

In [3]:
from dask.distributed import Client, LocalCluster, Queue

In [4]:
scheduler_file = 'scheduler.json'
cluster = LocalCluster(n_workers=1, threads_per_worker=1, host="0.0.0.0")
client = Client(cluster)
client.write_scheduler_file(scheduler_file)

We can now verify our scheduler is up and running. We also created a `scheduler.json` file that can be used to connect as many notebooks as desired.

In [5]:
client

Client Scheduler: tcp://128.55.224.44:38519,Cluster Workers: 1 Cores: 1 Memory: 540.15 GB


In [6]:
import os 

if os.path.isfile(scheduler_file):
    print("Scheduler file with name {} exists!".format(scheduler_file))

Scheduler file with name scheduler.json exists!


Now that client is up and running, there are potentially two ways to submit and gather data from a PSANA job:

1. Using a `Queue` in Dask. 
2. Using Pub/Sub scheme. 

## Queue example

In [7]:
!python /global/common/software/lcls/psana/lcls2/psana/psana/tests/setup_input_files.py


Start writing offsets.
Finished writing smd for 19 events with size (B): 39136

Start writing offsets.
Finished writing smd for 19 events with size (B): 34744


In [ ]:
# %env PS_SRV_NODES=2
# %srun -n 3 -A m3384  -C haswell -J smallD -q debug  -o example.log python example.py

In [8]:
%sbatch example.sub

'Submitted batch job 28406816\n'

In [9]:
%squeue -u melkhati

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,28406816,debug_hsw,smallD,melkhati,R,0:04,1,nid00632


In [10]:
q = Queue("psana")

In [11]:
q.qsize()

10

In [12]:
import numpy as np

results = []
for data in range(q.qsize()):
    results.append(np.array(q.get()))

In [13]:
results = np.array(results)
results

array([[1, 1],
       [1, 1],
       [2, 2],
       [2, 2],
       [3, 3],
       [3, 3],
       [4, 4],
       [4, 4],
       [5, 5],
       [5, 5]])